In [2]:
#图像处理完整样例
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#调整图像亮度、对比度、色饱和度 调整顺序不同会影响最后得到的结果
def distort_color(image,color_ordering=0):
    #参数 color_ordering参数代表的是各种操作的顺序
    if color_ordering==0:
        image=tf.image.random_brightness(image,max_delta=32./255.)#亮度
        image=tf.image.random_saturation(image,lower=0.5,upper=1.5)#饱和度
        image=tf.image.random_hue(image,max_delta=0.2)
        image=tf.image.random_contrast(image,lower=0.5,upper=1.5)#对比度